In [8]:
# Simple random 0/1 generator
# Just Hadamard and measure

from projectq import MainEngine
from projectq.ops import H, Measure, Ry

def myfunc(eng):
    qubit = eng.allocate_qubit()
    
    H | qubit
    # Ry(-3.1416/2) | qubit
    Measure | qubit
    eng.flush()
    return(int(qubit))

eng = MainEngine()
j=0
for i in range(1,100):
    j+=myfunc(eng)
    
print("#1s: {}".format(int(j)))

#1s: 41


In [11]:
# Draw a circuit for Bell-pair preparation

from projectq import MainEngine 
from projectq.ops import H, CNOT
from projectq.backends import CircuitDrawer

def create_bell_pair(eng):
    b1 = eng.allocate_qubit()
    b2 = eng.allocate_qubit()
    
    H | b1
    CNOT | (b1,b2)
    
    return b1, b2

# create the main compiler engine 
drawing_engine = CircuitDrawer()
eng = MainEngine(drawing_engine)

create_bell_pair(eng)

eng.flush()
print(drawing_engine.get_latex())


\documentclass{standalone}
\usepackage[margin=1in]{geometry}
\usepackage[hang,small,bf]{caption}
\usepackage{tikz}
\usepackage{braket}
\usetikzlibrary{backgrounds,shadows.blur,fit,decorations.pathreplacing,shapes}

\begin{document}
\begin{tikzpicture}[scale=0.8, transform shape]

\tikzstyle{basicshadow}=[blur shadow={shadow blur steps=8, shadow xshift=0.7pt, shadow yshift=-0.7pt, shadow scale=1.02}]\tikzstyle{basic}=[draw,fill=white,basicshadow]
\tikzstyle{operator}=[basic,minimum size=1.5em]
\tikzstyle{phase}=[fill=black,shape=circle,minimum size=0.1cm,inner sep=0pt,outer sep=0pt,draw=black]
\tikzstyle{none}=[inner sep=0pt,outer sep=-.5pt,minimum height=0.5cm+1pt]
\tikzstyle{measure}=[operator,inner sep=0pt,minimum height=0.5cm, minimum width=0.75cm]
\tikzstyle{xstyle}=[circle,basic,minimum height=0.35cm,minimum width=0.35cm,inner sep=-1pt,very thin]
\tikzset{
shadowed/.style={preaction={transform canvas={shift={(0.5pt,-0.5pt)}}, draw=gray, opacity=0.4}},
}
\tikzstyle{swapstyle}=[inne

In [16]:
# Implementing the teleportation protocol

from projectq import MainEngine
from projectq.backends import CircuitDrawer
from projectq.ops import All, H, CNOT, Measure, Rz, X, Z
from projectq.meta import Dagger, Control 

def create_bell_pair(eng):
    b1 = eng.allocate_qubit()
    b2 = eng.allocate_qubit()
    
    H | b1
    CNOT | (b1, b2)
    
    return b1, b2

def run_teleport(eng, state_preparation_map, verbose=False):
    # first make a Bell pair
    b1, b2 = create_bell_pair(eng)
    
    # let Alice prepare her state
    psi = eng.allocate_qubit()
    if verbose:
        print("Alice is preparing her state from scratch, i.e. |0>...")
    state_preparation_map(eng, psi)
    
    # entangle Alice's state with her Bell qubit b1
    CNOT | (psi, b1)
    if verbose:
        print("Alice has entangled her state with her side of the Bell.")
        
    # rotate to Hadamard basis, measure psi and b1
    H | psi
    Measure | psi
    Measure | b1
    msg_to_bob = [int(psi),int(b1)]
    if verbose:
        print("Alice is sending msg {} to Bob".format(msg_to_bob))
        
    # Bob makes his corrections
    with Control(eng, b1):
        X | b2
    with Control(eng, psi):
        Z | b2
        
    # Bob tries to uncompute the b2 state using the psi creator
    if verbose:
        print("Bob is trying to uncompute the state he received...")
    with Dagger(eng):
        state_preparation_map(eng, b2)
    
    # Check if the uncompute was successful
    # The simulator only allows deleting qubits in computational basis states
    del b2
    eng.flush()
    
    if verbose:
        print("Bob successfully arrived at state |0> after uncomputing")
        
    
if __name__ == "__main__":
    # Create main compiler engine with a simulator backend
    # eng = MainEngine()
    
    # Create main engine with a circuit drawer backend 
    drawing_engine = CircuitDrawer()
    eng = MainEngine(drawing_engine)
    
    # define state preparation map for the state to be teleported
    # it takes |0> as input
    # Alice uses it to prepare a state and teleport it to bob
    # Bob uses it to uncompute the received state
    
    def create_state(eng, qb):
        H | qb
        Rz(1.21) | qb
        
    # run the teleportation protocol and let Bob uncompute his qubit
    run_teleport(eng, create_state, verbose=False)
    
    
    print(drawing_engine.get_latex())
        

        

\documentclass{standalone}
\usepackage[margin=1in]{geometry}
\usepackage[hang,small,bf]{caption}
\usepackage{tikz}
\usepackage{braket}
\usetikzlibrary{backgrounds,shadows.blur,fit,decorations.pathreplacing,shapes}

\begin{document}
\begin{tikzpicture}[scale=0.8, transform shape]

\tikzstyle{basicshadow}=[blur shadow={shadow blur steps=8, shadow xshift=0.7pt, shadow yshift=-0.7pt, shadow scale=1.02}]\tikzstyle{basic}=[draw,fill=white,basicshadow]
\tikzstyle{operator}=[basic,minimum size=1.5em]
\tikzstyle{phase}=[fill=black,shape=circle,minimum size=0.1cm,inner sep=0pt,outer sep=0pt,draw=black]
\tikzstyle{none}=[inner sep=0pt,outer sep=-.5pt,minimum height=0.5cm+1pt]
\tikzstyle{measure}=[operator,inner sep=0pt,minimum height=0.5cm, minimum width=0.75cm]
\tikzstyle{xstyle}=[circle,basic,minimum height=0.35cm,minimum width=0.35cm,inner sep=-1pt,very thin]
\tikzset{
shadowed/.style={preaction={transform canvas={shift={(0.5pt,-0.5pt)}}, draw=gray, opacity=0.4}},
}
\tikzstyle{swapstyle}=[inne